<a href="https://colab.research.google.com/github/eric-r-xu/DiscardWisdom/blob/main/HKMahjong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from collections import defaultdict
import matplotlib.pyplot as plt

def show_image(img, scale):
    resized_img = cv2.resize(
        img, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA
    )
    # Convert BGR to RGB for displaying with matplotlib
    rgb_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb_img)
    plt.axis('off')
    plt.show()

def init_screenshot(image_path):
    width, height = 2778, 1284
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Error: Could not read the target image.")

    # Resize the image to the specified dimensions
    resized_image = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    # Initialize a mask with the new dimensions
    mask = np.zeros((height, width), dtype=bool)

    return resized_image, mask

def find_and_print_locations(template_image, image, match_mask):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    original_template_gray = cv2.cvtColor(template_image, cv2.COLOR_BGR2GRAY)
    threshold = 0.8
    locations = 0

    resized_template = original_template_gray
    result = cv2.matchTemplate(image_gray, resized_template, cv2.TM_CCOEFF_NORMED)
    matches = np.where(result >= threshold)

    for y, x in zip(*matches):
        end_y, end_x = y + resized_template.shape[0], x + resized_template.shape[1]
        if np.any(match_mask[y:end_y, x:end_x]):
            continue
        match_mask[y:end_y, x:end_x] = True
        cv2.rectangle(image, (x, y), (end_x, end_y), (0, 255, 255), -1)
        locations += 1

    return locations

# Paths to your images (ensure these paths are correct in your Colab environment)
screenshot_path = "/content/drive/MyDrive/mahjong/examples/1.png"
template_dir = "/content/drive/MyDrive/mahjong/hkmj_tile_templates/"

# Uploading and mounting your files on Google Colab
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

template_files = [
    os.path.join(root, file)
    for root, dirs, files in os.walk(template_dir)
    for file in files
    if file.endswith((".PNG", ".png"))
]

image, match_mask = init_screenshot(screenshot_path)
templates = {file: cv2.imread(file) for file in template_files}

tiles_found = defaultdict(int)
collective = defaultdict(int)
player_you = defaultdict(int)

for template_path, template_image in templates.items():
    _template_id = template_path.split("/")[-3:]
    _player, _type, _tile = (
        _template_id[0],
        _template_id[1],
        _template_id[2].replace(".png", ""),
    )
    if template_image is None:
        print(f"Error: Could not read the template image at {template_path}")
        continue
    found_tiles = find_and_print_locations(template_image, image, match_mask)
    if found_tiles > 0:
        tiles_found[_tile] += found_tiles
        if _player == 'player_you':
            player_you[_tile] += found_tiles
        if _player == 'collective':
            collective[_tile] += found_tiles

# Create a semi-transparent overlay
overlay = image.copy()
highlight_color = (0, 255, 255)  # Yellow color for highlights

# Redraw the rectangles on the overlay
for template_path, template_image in templates.items():
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    original_template_gray = cv2.cvtColor(template_image, cv2.COLOR_BGR2GRAY)
    threshold = 0.8
    resized_template = original_template_gray
    result = cv2.matchTemplate(image_gray, resized_template, cv2.TM_CCOEFF_NORMED)
    matches = np.where(result >= threshold)

    for y, x in zip(*matches):
        end_y, end_x = y + resized_template.shape[0], x + resized_template.shape[1]
        if np.any(match_mask[y:end_y, x:end_x]):
            continue
        match_mask[y:end_y, x:end_x] = True
        cv2.rectangle(overlay, (x, y), (end_x, end_y), highlight_color, -1)


highlight_image = cv2.addWeighted(overlay, 0.5, image, 1, 0)


show_image(highlight_image, 0.3)

print("Finished tile recognition of yellow highlighted tiles above")



class MahjongHandChecker:
    def __init__(self, tiles, all_tiles):
        self.tiles = (
            tiles.copy()
        )
        self.all_tiles = all_tiles.copy()

    def is_pung(self, tile):
        """Check if there's a pung of the given tile."""
        return self.tiles[tile] >= 3

    def is_chow(self, tile):
        """Check if there's a chow starting with the given tile."""
        if tile[-1] in ["b", "d", "c"]:  # Ensure the tile is a suited tile
            # print(f'tile = {tile}')
            num = int(tile[:-1])
            suit = tile[-1]
            # Check the sequence exists
            return all(self.tiles.get(f"{num + i}{suit}", 0) >= 1 for i in range(3))
        return False

    def is_pair(self, tile):
        """Check if there's a pair of the given tile."""
        return self.tiles[tile] >= 2

    def remove_set(self, tile):
        """Remove a pung or chow starting with the given tile, favoring pung first."""
        if self.is_pung(tile):
            self.tiles[tile] -= 3
            return True
        elif self.is_chow(tile):
            num = int(tile[:-1])
            suit = tile[-1]
            for i in range(3):
                self.tiles[f"{num + i}{suit}"] -= 1
            return True
        return False

    def remove_pair(self, tile):
        """Remove a pair of the given tile."""
        self.tiles[tile] -= 2

    def is_all_honors(self):
        """Check if the hand consists entirely of honor tiles (winds and dragons)."""
        honor_tiles = {"ewh", "swh", "wwh", "nwh", "rdh", "gdh", "wdh"}
        for tile in self.tiles:
            if self.tiles[tile] > 0 and tile not in honor_tiles:
                return False
        return True

    def is_thirteen_orphans(self):
        """Check if the hand is a valid Thirteen Orphans hand."""
        required_tiles = {
            "1b",
            "9b",
            "1c",
            "9c",
            "1d",
            "9d",
            "ewh",
            "swh",
            "wwh",
            "nwh",
            "rdh",
            "gdh",
            "wdh",
        }
        found_pair = False
        for tile in required_tiles:
            if self.tiles.get(tile, 0) == 0:
                return False
            if self.tiles.get(tile, 0) > 1:
                found_pair = True
        return found_pair

    def is_seven_pairs(self):
        """Check if the hand consists of exactly seven different pairs."""
        if sum(value == 2 for value in self.tiles.values()) == 7:
            return True
        return False

    def is_all_flowers(self):
        """Check if the hand contains all eight flower tiles."""
        flower_season_tiles = {"1f", "2f", "3f", "4f"}
        return all(self.tiles.get(tile, 0) > 1 for tile in flower_season_tiles)

    def check_special_hands(self):
        """Check for any special hands."""
        if self.is_all_honors():
            print("All Honors")
            return True
        elif self.is_thirteen_orphans():
            print("Thirteen Orphans")
            return True
        elif self.is_seven_pairs():
            print("Seven Pairs")
            return True
        elif self.is_all_flowers():
            print("All Flowers")
            return True
        return False

    def check_winning_hand(self, sets_found=0, pair_found=False):
        if sets_found == 4 and pair_found:
            return True  # Found 4 sets and a pair

        # Try to find a pair if not found yet
        if not pair_found:
            for tile in list(self.tiles):
                if self.is_pair(tile):
                    self.remove_pair(tile)
                    if self.check_winning_hand(sets_found, True):
                        return True
                    self.tiles[tile] += 2  # Backtrack

        # Try to find sets
        for tile in list(self.tiles):
            if self.tiles[tile] > 0 and self.remove_set(tile):
                if self.check_winning_hand(sets_found + 1, pair_found):
                    return True
                # Backtrack
                num = int(tile[:-1])
                suit = tile[-1]
                if self.is_pung(tile):
                    self.tiles[tile] += 3
                else:
                    for i in range(3):
                        self.tiles[f"{num + i}{suit}"] += 1

        return False

    def suggest_discard(self):
        potential_discards = {}
        for tile in self.tiles:
            if self.tiles[tile] > 0 and tile[-1] != 'f': # flowers can't be discarded
                # Temporarily remove the tile and evaluate the hand
                original_count = self.tiles[tile]
                self.tiles[tile] -= 1
                potential_discards[tile] = self.evaluate_hand_potential()
                self.tiles[tile] = original_count  # Restore the original tile count

        # Find the tile whose removal minimizes the potential for a winning hand
        least_useful_tile = min(potential_discards, key=potential_discards.get, default=None)
        return least_useful_tile, potential_discards[least_useful_tile] if least_useful_tile else None, potential_discards

    def evaluate_hand_potential(self):
        # Calculate the potential of the hand based on remaining tiles and current hand composition
        score = 0
        for tile, count in self.tiles.items():
            if count >= 3:
                score += 1  # Potential pung already in hand
            elif count == 2 and (self.all_tiles[tile] - count > 0):
                score += 0.5  # Potential pung if one more tile can be drawn

            # Check for potential chows for suited tiles
            if tile[-1] in ['b', 'c', 'd'] and count > 0:
                num = int(tile[:-1])
                suit = tile[-1]
                # Check for potential chows by looking forward and backward one and two tiles
                chow_potentials = [
                    (num-2, num-1, num),
                    (num-1, num, num+1),
                    (num, num+1, num+2)
                ]
                for triplet in chow_potentials:
                    if all((self.all_tiles.get(f"{n}{suit}", 0) - self.tiles.get(f"{n}{suit}", 0)) > 0 for n in triplet if n >= 1 and n <= 9):
                        score += 1  # Possible chow with available tiles

        return score


def initialize_all_tiles():
    # Initialize dictionary to hold the count of all tiles in a Mahjong set
    all_tiles = {}

    # Adding 4 copies of each suit tile (bamboos, dots, characters)
    for num in range(1, 10):
        for suit in [
            "b",
            "d",
            "c",
        ]:  # 'b' for bamboos, 'd' for dots, 'c' for characters
            all_tiles[f"{num}{suit}"] = 4

    # Adding 4 copies of each wind and dragon tile
    for tile in ["nwh", "swh", "wwh", "ewh", "gdh", "rdh", "wdh"]:  # winds and dragons
        all_tiles[tile] = 4

    # Adding 2 copies of each flower tile
    for num in range(1, 5):
        all_tiles[f"{num}f"] = 2

    return all_tiles


# Initialize all tiles
all_tiles = initialize_all_tiles()

# Initialize dictionaries for your tiles and opponent's tiles
your_tiles = defaultdict(int)


for tile, num_tiles in collective.items():
    all_tiles[tile] -= num_tiles  # decrement from all_tiles

checker = MahjongHandChecker(player_you, all_tiles)

least_useful_tile, least_useful_score, all_discards = checker.suggest_discard()
print("\nPotential scores if each tile is discarded:\n")
for tile, score in sorted(all_discards.items()):
    print(f"{tile}: {score}")


Mounted at /content/drive
